### Add Directories/ Load Files

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utils import find_folders
import mne
from scipy.stats import mannwhitneyu
import math
import funs_logistic_regression
from importlib import reload
import pickle

import dat_preproc

In [ ]:
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
path_raw_accel = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\with_med_FTG\\files\\raw_accel_files'

Handle TMSI Accelerometer Data

In [ ]:
# RUN THIS CODE
import tmsi_poly5reader

poly5_name = 'Sub-029_18mfu_dysk_segNoEnt_ramp-20230216T104449.DATA.Poly5'

# LOAD FILE
accel = tmsi_poly5reader.Poly5Reader(
    os.path.join(path_raw_accel,
                 poly5_name)
)

raw_acc = accel.read_data_MNE()
dn_accel = raw_acc.resample(250, npad="auto")


In [ ]:
dn_accel.ch_names


In [ ]:
my_dn_accel = dn_accel.get_data(picks=[6,7,8])
my_dn_accel.shape
plt.plot(my_dn_accel[0,:])
plt.show()


In [ ]:
np.savetxt(os.path.join(
    path_raw_accel,
    'Sub029_M1_RampUp_250Hz_LHAND.csv'
), my_dn_accel, delimiter = ',')

In [ ]:
accel_blocks_df = pd.read_excel(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'Accel_Blocks_NoEntrainment.xlsx'
))

accel_blocks_df.head(10)

In [ ]:
features_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\subcohort_noENTRAINMENT\\retap_results\\features'
blocks_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\subcohort_noENTRAINMENT\\retap_results\\extracted_tapblocks'

In [ ]:
imp_features = ['freq', 'mean_raise_velocity', 'coefVar_raise_velocity', 'trace_RMSn', 'coefVar_intraTapInt', 'slope_intraTapInt','trace_entropy','jerkiness_trace', 'mean_tapRMS',
                'coefVar_tapRMS', 'mean_impactRMS', 'coefVar_impactRMS', 'slope_impactRMS', 'coefVar_tap_entropy', 'slope_tap_entropy']

In [ ]:
all_subs = accel_blocks_df['Percept_ID'].unique()

for sub in all_subs:
    print(sub)

In [ ]:
conditions = [1,4]

for sub in all_subs:

    this_sub = sub

    for cond in conditions:

        this_cond = cond

        these_blocks = accel_blocks_df.loc[(accel_blocks_df['Percept_ID'] == this_sub) & (accel_blocks_df['Cond'] == this_cond), 'Blocks']
        block_strings = ['block' + str(num) for num in these_blocks]
        print(block_strings)

        # Filter the json files based on conditions
        jsons_to_import = []
        for filename in os.listdir(features_path):
            if filename.endswith('.json') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}.json'):
                        jsons_to_import.append(filename)

        print(jsons_to_import)

        # Filter the csv files based on conditions
        csvs_to_import = []
        for filename in os.listdir(blocks_path):
            if filename.endswith('.csv') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}_250Hz.csv'):
                        csvs_to_import.append(filename)

        print(csvs_to_import)

        # Read the selected JSON files

        combined_dict = []
        for filename in jsons_to_import:
            file_path = os.path.join(features_path, filename)
            with open(file_path) as f:
                this_block_feat = json.load(f)
                matching_keys = set(imp_features) & set(this_block_feat.keys())
                combined_dict.append({key: this_block_feat[key] for key in matching_keys})

        #SAVE IT
        suptitle = str(this_sub) + ' - Condition ' + str(this_cond)

        file_name = "".join(suptitle.split()) + '_features.json'

        #PLOT IT
        %matplotlib qt
        fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

        suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
        fig.suptitle(suptitle, fontsize=14, fontweight='bold')

        rounded_list = [
            {key: round(value, 2) for key, value in dictionary.items()}
            for dictionary in combined_dict
        ]

        for num, file in enumerate(csvs_to_import):
            this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
            axs[num].plot(this_block_csv)
            axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
            axs[num].set_title(rounded_list[num])

            plt.savefig(os.path.join(
            project_path,
            'results',
            'accelerometer',
            'inspections',
            'group_NO_Entrainment',
            "".join(suptitle.split()),
            ), dpi = 250
            )
                
            # Combine the file name with the current directory to create the file path
            file_path = os.path.join(
                project_path,
                'results',
                'accelerometer',
                'inspections',
                'group_NO_Entrainment')

            # Serializing json
            json_object = json.dumps(combined_dict, indent=4)
            
            # Writing to sample.json
            with open(os.path.join(file_path,file_name), "w") as outfile:
                outfile.write(json_object)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
fig.suptitle(suptitle, fontsize=14, fontweight='bold')

rounded_list = [
    {key: round(value, 2) for key, value in dictionary.items()}
    for dictionary in combined_dict
]

for num, file in enumerate(csvs_to_import):
    this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
    axs[num].plot(this_block_csv)
    axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
    axs[num].set_title(rounded_list[num])
    

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'inspections',
    "".join(suptitle.split()),
), dpi = 250
)

### Average all blocks for each condition within subject

In [ ]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections', 'Condition3'
)

all_features_cond1 = pd.DataFrame()

# Loop through the JSON files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        percept_id = filename[:6]

        # Load the JSON file
        with open(file_path, 'r') as f:
            json_data = json.load(f)
        
        # Initialize a dictionary to store the accumulated sums and counts for each key
        summed_values = {}
        count_values = {}
        
        # Iterate through the dictionaries in the JSON data
        for data_dict in json_data:
            # Accumulate the sums and counts for each key
            for key, value in data_dict.items():
                if key not in summed_values:
                    summed_values[key] = value
                    count_values[key] = 1
                else:
                    summed_values[key] += value
                    count_values[key] += 1
        
        # Calculate the averages for each key
        #averaged_values = {key: summed_values[key] / count_values[key] for key in summed_values}
        averaged_values = {key: summed_values[key] for key in summed_values}
        averaged_values['Percept_ID'] = percept_id
        
        # Append the averaged values to the dataframe
        all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)




In [ ]:
# Print the resulting dataframe
all_features_cond1 = all_features_cond1.reindex(columns=['Percept_ID'] + list(all_features_cond1.columns[:-1]))
print(all_features_cond1)

In [ ]:
# Save the dataframe as JSON
json_file_path = os.path.join(json_dir, 'all_features_cond3.json')
all_features_cond1.to_json(json_file_path, orient='records')

### Add all blocks together without averaging them

In [ ]:
json_dir = os.path.join(project_path, 'results', 'accelerometer', 'MotorPerf_Comps')

filenames = os.listdir(json_dir)
json_filenames = [filename for filename in filenames if not filename.startswith('.DS_Store')]

data_frames = []
for filename in json_filenames:
    file_path = os.path.join(json_dir, filename)
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        df.insert(0, 'Percept_ID',filename[:6])   # Extract filename without extension
        data_frames.append(df)

# Concatenate all DataFrames into a single big DataFrame
big_dataframe = pd.concat(data_frames, ignore_index=True)

In [ ]:
json_file_path = os.path.join(json_dir, 'all_featureswithENTR_cond4_ALLBLOCKS.json')
big_dataframe.to_json(json_file_path, orient='records')

### Plot all averaged values in four conditions

In [ ]:
'''json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections\\'
)'''

json_dir = os.path.join(project_path, 'results', 'accelerometer', 'inspections')

feat_cond_1 = pd.read_json(os.path.join(json_dir,'all_features_cond1.json'))
feat_cond_2 = pd.read_json(os.path.join(json_dir,'all_features_cond2.json'))
feat_cond_3 = pd.read_json(os.path.join(json_dir,'all_features_cond3.json'))
feat_cond_4 = pd.read_json(os.path.join(json_dir,'all_features_cond4.json'))


fts_of_interest = ['Percept_ID',"coefVar_intraTapInt", "trace_RMSn", "coefVar_impactRMS", "mean_raise_velocity"]
feat_cond_1 = feat_cond_1[fts_of_interest]
feat_cond_2 = feat_cond_2[fts_of_interest]
feat_cond_3 = feat_cond_3[fts_of_interest]
feat_cond_4 = feat_cond_4[fts_of_interest]

#feat_cond_4 = pd.read_json(str(json_dir) + 'all_features_cond4.json')


In [ ]:
# Get the column values as NumPy arrays
column1_values = feat_cond_1['Percept_ID'].values
column2_values = feat_cond_2['Percept_ID'].values
column3_values = feat_cond_3['Percept_ID'].values
column4_values = feat_cond_4['Percept_ID'].values

# Find the similar values between the three columns
similar_values = np.intersect1d(column1_values, np.intersect1d(column2_values, column3_values))
print(similar_values)

# Keep only the rows with similar values in each dataframe
feat_cond_1_filtered = feat_cond_1[feat_cond_1['Percept_ID'].isin(similar_values)]
feat_cond_2_filtered = feat_cond_2[feat_cond_2['Percept_ID'].isin(similar_values)]
feat_cond_3_filtered = feat_cond_3[feat_cond_3['Percept_ID'].isin(similar_values)]
feat_cond_4_filtered = feat_cond_4[feat_cond_4['Percept_ID'].isin(similar_values)]

In [ ]:
#Sub-analysis four conditions within subjects:
# Add 'Condition' column to each dataframe
feat_cond_1['Condition'] = 1
feat_cond_2['Condition'] = 2
feat_cond_3['Condition'] = 3
feat_cond_4['Condition'] = 4
trial_df = pd.concat([feat_cond_1, feat_cond_2, feat_cond_3, feat_cond_4], axis=0, ignore_index=True)
trial_df

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 6), sharey=False)
merged_df = trial_df
ylabels = ['Coef. Variation. Intra Tap Int','Normalized RMS','Coef. Variation. Impact RMS','Mean Raise Velocity']
# Iterate over columns and plot boxplots and scatters
for i, column in enumerate(['coefVar_intraTapInt', 'trace_RMSn', 'coefVar_impactRMS', 'mean_raise_velocity']):
    for condition in range(1, 5):  # Consider pairs of adjacent conditions
        condition_data_curr = merged_df[merged_df['Condition'] == condition]
        condition_data_next = merged_df[merged_df['Condition'] == condition + 1]

        # Plot boxplot
        axes[i].boxplot(condition_data_curr[column], positions=[condition], widths=0.5, showfliers=False)

        # Plot scatters and connect with lines to the next condition
        x_values_curr = [condition] * len(condition_data_curr[column])
        x_values_next = [condition + 1] * len(condition_data_next[column])

        axes[i].scatter(x_values_curr, condition_data_curr[column], color='C{}'.format(condition - 1), s=20)
        axes[i].scatter(x_values_next, condition_data_next[column], color='C{}'.format(condition), s=20)

        for x_curr, x_next, y_curr, y_next in zip(x_values_curr, x_values_next, condition_data_curr[column], condition_data_next[column]):
            axes[i].plot([x_curr, x_next], [y_curr, y_next], color='grey'.format(condition - 1), alpha=0.3)

    # Set labels and title for each subplot
    axes[i].set_xticks([1, 2, 3, 4])
    xticklabels = ['DBS Off', 'DBSOn-PreEntrain', 'DBSOn-FirstEtrain', 'DBSOn-HighestEntrain']
    axes[i].set_xticklabels(xticklabels)
    axes[i].set_xticklabels(xticklabels, rotation = 45)
    axes[i].set_ylabel(ylabels[i])

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig(os.path.join(project_path, 'results', 'accelerometer', 'inspections','Off-OnPreLowHigh_4conds'),dpi=300)

In [ ]:
feat_cond_1['Cohort'] = 1
feat_cond_2['Cohort'] = 2
lme_dataframe = pd.concat([feat_cond_1,feat_cond_2], ignore_index=True)
lme_dataframe.to_excel('LME_MotorPerformance.xlsx')

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
lme_dataframe = pd.read_excel(os.path.join(project_path, 'results', 'accelerometer', 'MotorPerf_Comps','LME_MotorPerformance.xlsx'))
lme_dataframe

In [ ]:
md = smf.mixedlm("mean_raise_velocity ~ Stim", lme_dataframe, groups=lme_dataframe["Percept_ID"])
mdf = md.fit()
print(mdf.summary())

In [ ]:
%matplotlib qt
#import mlxtend
from scipy.stats import mannwhitneyu

# Get the column names excluding 'Percept_ID'
columns = [col for col in feat_cond_1.columns if col != 'Percept_ID']

#fig, axes = plt.subplots(1, len(columns), figsize=(25, 5))  # Adjust figsize as needed

fig, axes = plt.subplots(1, 4, figsize=(10, 3))
axes = axes.flatten()
# Iterate over each column and create a boxplot for each dataframe
for i, column in enumerate(columns):
    # Get the data for the column from each dataframe
    data1 = feat_cond_1[column]
    data2 = feat_cond_2[column]
    data3 = feat_cond_3[column]
    data4 = feat_cond_4[column]

    #p_value = mlxtend.permutation_test(data1.values, data2.values, paired=False, method="approximate", seed=0, num_rounds=100000)

    #p_value_round = np.round(p_value, decimals = 6)

    statistic, p_value = mannwhitneyu(data1, data2)
    p_value_round = np.round(p_value, decimals = 6)


     # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2, data3, data4], widths = 0.5, showfliers=False)
    axes[i].scatter([1]*len(data1), data1, color='dimgrey', s =20)
    axes[i].scatter([2]*len(data2), data2, color='darkred', s=20)
    axes[i].scatter([3]*len(data3), data3, color='green', s=20)
    axes[i].scatter([4]*len(data4), data4, color='blue', s=20)

    axes[i].set_title(column)


    # Create a boxplot for the column in the corresponding subplot
    #axes[i].set_title(column + ':' + str(p_value_round))

# Add a legend at the bottom of the plot
legend_labels = ['M1S0', 'M1S1:PreSubharmonic', 'M1S1:Subh-First Amp','M1S1:Subh-Highest Amp']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['dimgrey', 'darkred', 'green','blue']]

#legend_labels = ['GroupNO', 'GroupWith']
#legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['dimgrey', '#DDCC77']]

fig.legend(legend_markers, legend_labels, loc='lower center', ncol=3)

# Adjust spacing between subplots and legend
fig.tight_layout(rect=[0, 0.1, 1, 1])

In [ ]:
plt.savefig(os.path.join(
    json_dir, 'All_Blocks_GROUPSCOMPARISONS_WITH_WITHOUT_Wilcoxon'
), dpi = 400
)

plt.savefig(os.path.join(
    json_dir, 'All_Blocks_GROUPSCOMPARISONS_WITH_WITHOUT_Wilcoxon.pdf'
))

## Compare ephys during movement and subharmonic

#### Fix Package Loss in Json Files

In [ ]:
import json
file_Name = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PERCEPT_MOTHERFOLDER\\RawFiles\\sub-028\\12MFU\\Dyskinesia\\Report_Json_Session_Report_20220825T131757.json'

with open(file_Name, 'r') as file:
    # Load the JSON data
    data = json.load(file)

In [ ]:
len(data['BrainSenseTimeDomain'][0]['TimeDomainData'])/250/60

In [ ]:
import re
ticks = data['BrainSenseTimeDomain'][0]['TicksInMses']
ticks_n = np.array(re.findall(r'\d+', ticks), dtype=int)
print(ticks_n)
'''diff = np.diff(ticks_n)
indices = np.where(diff > 250)[0] + 1
print(indices)'''


In [ ]:
ticks_n.shape

#### Synchronize Accel data

In [ ]:
fif_path = os.path.join(project_path, 'data', 'Fifs', 'with_med_FTG')
accel_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all'
accel_fileName = 'Sub029_M1_RampUp_250Hz_LHAND.csv'

dn_accel = pd.read_csv(os.path.join(
    accel_path,
    accel_fileName
), header = None
)

Draw = mne.io.read_raw_fif(os.path.join(fif_path,'Sub029_Ramp125_FIF.fif'))

In [ ]:
%matplotlib qt

time_points = np.arange(dn_accel.shape[1]) / 250

for i in range(3):
    plt.plot(time_points, dn_accel.iloc[i,:])

# Show the modified figure
plt.show()


In [ ]:
print(f'Number of Samples of Accel data = {dn_accel.shape[1]}')
print(f'Number of Samples of Raw data = {Draw.n_times}')

In [ ]:
num_samples_start = 34.510 *250
num_samples_start

In [ ]:
dn_accel_array = dn_accel.values

#num_samples_start = 23.767 *250
#data_trimmed = dn_accel_array[:, 8627:]
#data_trimmed.shape

data_trimmed = data_trimmed[:, :-48582]
data_trimmed.shape


In [ ]:
data_trimmed.shape[1] - Draw.n_times

In [ ]:
Draw.ch_names

In [ ]:
stim = Draw.get_data(picks = 'STIM_R_125Hz_60us')[0,:]
plt.plot(data_trimmed[0,:])
plt.plot(stim)

In [ ]:
np.save(os.path.join(
    accel_path,
    'Sub029_M1_RampUp_cut_traces.npy'
), data_trimmed)

#### Define Rest vs Movement

In [ ]:
SUBID = 'Sub005_RampUp'

#define your paths
fif_path = os.path.join(project_path, 'data', 'Fifs', 'with_med_FTG')
FFTs_path = os.path.join(project_path, 'data', 'FFTs', 'with_med_FTG')
cut_traces_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all\\Synced_traces\\'

#Import your data
accel = np.load(os.path.join(cut_traces_path, 'Sub005_cut_traces.npy'))

#accel = np.genfromtxt(os.path.join(cut_traces_path, 'Sub005_cut_traces.csv'), delimiter=',')

Draw = mne.io.read_raw_fif(os.path.join(fif_path,'Sub005_FIF.fif'))
D = np.load(os.path.join(FFTs_path, 'sub005_FFT.npy'))

Draw.ch_names

In [ ]:
%matplotlib qt
time_points1 = np.arange(accel.shape[1]) / 250
time_points2 = np.arange(Draw.n_times) / 250
stim = Draw.get_data(picks = 'STIM_R_130Hz_60us')[0]

plt.plot(time_points1, accel[2,:])
plt.plot(time_points2, stim*10e5)

In [ ]:
epoch_df = pd.read_excel(os.path.join(project_path, 'documents', 'FTG_Global.xlsx'), sheet_name = 'MovementEpochs_DBSOff')
epoch_df

In [ ]:
row = 9

rest_on = epoch_df.loc[row,'Rest_on']
rest_off = epoch_df.loc[row,'Rest_off']
rest_dur = rest_off - rest_on

block1_on = epoch_df.loc[row,'Block1_on']
block1_off = epoch_df.loc[row,'Block1_off']
block1_dur = block1_off - block1_on

block2_on = epoch_df.loc[row,'Block2_on']
block2_off = epoch_df.loc[row,'Block2_off']
block2_dur = block2_off - block2_on

block3_on = epoch_df.loc[row,'Block3_on']
block3_off = epoch_df.loc[row,'Block3_off']
block3_dur = block3_off - block3_on


In [ ]:
import matplotlib.patches as patches
%matplotlib qt

time_points = np.arange(accel.shape[1]) / 250
fig, ax = plt.subplots()
plt.plot(time_points1, accel[2,:])
plt.plot(time_points2, stim)
ymin, ymax = plt.ylim()

rect1 = patches.Rectangle((rest_on, ymin), rest_dur, ymax, linewidth=1, facecolor='red', alpha = 0.3)
rect2 = patches.Rectangle((block1_on, ymin), block1_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)
rect3 = patches.Rectangle((block2_on, ymin), block2_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)
rect4 = patches.Rectangle((block3_on, ymin), block3_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)

plt.gca().add_patch(rect1)
plt.gca().add_patch(rect2)
plt.gca().add_patch(rect3)
plt.gca().add_patch(rect4)

plt.xlabel('Time [sec]')
plt.ylabel('Acceleration')

plt.title(f'{SUBID}')

plt.show()

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'STIM_OFF', f'{SUBID}_Epochs.jpg'),
    dpi=200)

In [ ]:
SIDE = 1

rest_ps = np.nanmean(D[SIDE, :, int(rest_on):int(rest_off)], 1)

tap1_ps = np.nanmean(D[SIDE, :, int(block1_on):int(block1_off)], 1)
tap2_ps = np.nanmean(D[SIDE, :, int(block2_on):int(block2_off)], 1)
tap3_ps = np.nanmean(D[SIDE, :, int(block3_on):int(block3_off)], 1)

In [ ]:
plt.plot(np.arange(1,127), rest_ps, label = 'Rest', lw = 2, linestyle = 'dashed')
plt.plot(np.arange(1,127), tap1_ps, label = 'TapBlock1', lw = 2)
plt.plot(np.arange(1,127), tap2_ps, label = 'TapBlock2', lw = 2)
plt.plot(np.arange(1,127), tap3_ps, label = 'TapBlock3', lw = 2)

plt.legend()
plt.xlim(40,90)
plt.ylim(0,0.2)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

plt.title(f'{SUBID}')


In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul','STIM_OFF', f'{SUBID}_PowerSpectra.jpg'),
    dpi=200)

In [ ]:
tap_array = np.array([tap1_ps, tap2_ps, tap3_ps])
tap_array.shape

In [ ]:
np.save(os.path.join(project_path, 'data','movement_modul', 'STIM_OFF',f'{SUBID}_RestPS.npy'), rest_ps)
np.save(os.path.join(project_path, 'data','movement_modul', 'STIM_OFF', f'{SUBID}_TapPS.npy'), tap_array)

### Average all taps and plot

In [ ]:
import glob
data_dir = os.path.join(project_path, 'data', 'movement_modul', 'STIM_OFF\\')

file_list = glob.glob(data_dir + '*TapPS.npy')

# Initialize an empty list to store the arrays
for file_path in file_list:
    array = np.load(file_path)
    averaged_array = np.mean(array, axis=0)
    print(averaged_array.shape)
    
    name = file_path.split('\\')[-1].split('_')[0]
    print(name)

    np.save(os.path.join(
        data_dir, f'{name}_AVG_TAP.npy'
    ), averaged_array)

In [ ]:
%matplotlib qt
subject_names = set()
data_dir = os.path.join(project_path, 'data', 'movement_modul', 'STIM_OFF\\')
# Collect subject names
for filename in os.listdir(data_dir):
    if filename.endswith(".npy"):
        subject_name = filename.split("_")[0]
        subject_names.add(subject_name)


all_rest = []
all_tap = []

# Load and plot matching pairs
for subject_name in sorted(subject_names):
    tap_file = os.path.join(data_dir, f"{subject_name}_AVG_TAP.npy")
    rest_file = os.path.join(data_dir, f"{subject_name}_RestPs.npy")

    if os.path.isfile(tap_file) and os.path.isfile(rest_file):
        tap_data = np.load(tap_file)
        rest_data = np.load(rest_file)
        print(subject_name)
        all_rest.append(rest_data)
        all_tap.append(tap_data)
        # Plotting code
        plt.figure()
        plt.plot(tap_data, label="Tap")
        plt.plot(rest_data, label="Rest")
        plt.title(f"Subject: {subject_name}")
        #plt.ylim(0,0.4)
        plt.xlim(5,40)
        plt.legend()
        plt.show()

        '''plt.savefig(os.path.join(
            data_dir,
            f"{subject_name}_MeanMod"
        ), dpi = 200)
'''

In [ ]:
all_rest_df = np.transpose(np.array(all_rest))
all_rest_df = pd.DataFrame(all_rest_df)
all_rest_df.columns = (sorted(subject_names))
all_rest_df

In [ ]:
all_tap_df = np.transpose(np.array(all_tap))
all_tap_df = pd.DataFrame(all_tap_df)
all_tap_df.columns = (sorted(subject_names))
all_tap_df

In [ ]:
data_dir = os.path.join(project_path, 'data', 'movement_modul', 'STIM_OFF\\')
#all_rest_df = pd.read_excel(os.path.join(data_dir,'AllRest_preTap.xlsx'),index_col=None)
#all_tap_df = pd.read_excel(os.path.join(data_dir,'AVG_TAP.xlsx'),index_col=None)
#all_tap_df
all_rest_df.to_excel(os.path.join(data_dir,'AllRest_preTap.xlsx'))
all_tap_df.to_excel(os.path.join(data_dir,'AVG_TAP.xlsx'))

In [ ]:
colDict = {}
for key in ['Sub005', 'Sub007']:
    colDict[key] = 64
for key in ['Sub021','Sub025','Sub028','Sub033','Sub050','Sub065']:
    colDict[key] = 62
for key in ['Sub029']:
    colDict[key] = 72

max_d = {'Percept_ID':all_tap_df.columns,
          'Max_Rest_Subh': max_values_df1,
          'Max_Tap_Subh': max_values_df2
}

max_df = pd.DataFrame(data = max_d)
max_df
#statistic, p_value = mannwhitneyu(max_values_df1, max_values_df2)
#p_value_round = np.round(p_value, decimals = 6)

#print(p_value_round)

max_df.to_excel(os.path.join(project_path,'data','movement_modul', 'MaxSubh_RestMove.xlsx'), index = False)

In [ ]:
#all_rest_gamma = all_rest_df.iloc[50:81]
#all_tap_gamma = all_tap_df.iloc[50:81]

#rest_smoothed = all_rest_gamma.rolling(window=3, min_periods=1).mean()
#tap_smoothed = all_tap_gamma.rolling(window=3, min_periods=1).mean()

#max_values_df1 = all_rest_gamma.max()
df2_normalized = all_tap_df.div(max_df['Max_Rest_Subh'].values)
df2_normalized.columns = all_tap_df.columns
df2_normalized.columns 

In [ ]:
# Columns to add NaN rows and shift values
columns_63 = ['Sub021', 'Sub025', 'Sub028','Sub033', 'Sub050', 'Sub065']
columns_73 = ['Sub029']

df2_normalized2 = df2_normalized
# Adding NaN rows and shifting values for specified columns
for column in columns_63:
    # Shift values down by the number of rows
    df2_normalized2[column] = df2_normalized[column].shift(2)

    # Add NaN rows to the beginning of the column
    df2_normalized.loc[:2-1, column] = None

for column1 in columns_73:
    df2_normalized2[column1] = df2_normalized2[column1].shift(-7)
    #df2_normalized2.loc[:10-1, column1] = None


df2_normalized2

In [ ]:
%matplotlib qt
mean_tap = np.nanmean(df2_normalized2, axis = 1)
sem_tap = stats.sem(df2_normalized2, axis = 1, nan_policy= 'omit')

plt.plot(np.arange(1,127), mean_tap, color = 'purple', lw = 2)
plt.plot(np.arange(1,127), df2_normalized2, color = 'purple', alpha = 0.18)
plt.fill_between(np.arange(1,127),mean_tap - sem_tap, mean_tap + sem_tap, color = 'purple', alpha=0.3, linestyle = 'dashed', lw = 1.5)
plt.axhline(y = 1, color = 'grey', linestyle = ':', alpha = 0.5)
plt.xlim(50,80)
plt.ylim(0,2.5)

In [ ]:
def plot_columns_with_transparency(dataframe):
    # Line color and transparency
    transparencies = np.linspace(0.3,1,9)

    # Create a figure and axes
    fig, ax = plt.subplots()

    # Plot columns with increasing transparency
    for i, column in enumerate(dataframe.columns):
            dataframe[column].plot(ax=ax, color = 'purple', alpha=transparencies[i], label=f'{column}', lw = 2.5)

    
    # Add labels and title
    ax.set_ylabel('LFP Power [Normalized to Rest]')
    # Add a legend
    ax.legend(loc='best')

    # Show the plot
    plt.show()

# Plot the DataFrame columns
plot_columns_with_transparency(df2_normalized)
plt.axhline(y = 1, color = 'grey', linestyle = ':', alpha = 0.3)
plt.xlim(50,80)
plt.xticks(np.arange(50,85,5), labels = ['-15','-10','-5','1:2\nEntrainment', '+5','+10','+15'])
plt.ylim(0,3)



In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Movement_Modulation_All'),
    dpi=200)

plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Movement_Modulation_All.pdf'))

#### Boxplots Gamma Sync Stim Off

In [ ]:
dir = os.path.join(
    project_path, 'data', 'movement_modul', 'STIM_OFF'
)

all_rest_df = pd.read_excel(os.path.join(dir, 'AllRest_preTap.xlsx'), index_col=None)
all_tap_df = pd.read_excel(os.path.join(dir, 'AVG_TAP.xlsx'), index_col=None)

mean_rest = all_rest_df.iloc[39:89].mean(axis = 0)
mean_tap = all_tap_df.iloc[39:89].mean(axis = 0)

In [ ]:
# Sample array of 9 strings
labels_array = list(all_tap_df.columns.values)

# Create a figure and axis
fig, ax = plt.subplots()

# Boxplot
plt.boxplot([mean_rest, mean_tap], widths=0.3, showfliers=False, labels=['Rest', 'Tap'])

# Define a colormap with 9 distinct colors
colors = plt.cm.viridis(np.linspace(0, 1, 9))

# Scatter plot for mean_rest and mean_tap with unique colors and labels
for i in range(len(mean_rest)):
    ax.scatter([1, 2], [mean_rest[i], mean_tap[i]], color=colors[i], s=60, label=labels_array[i])

# Connecting lines
for x1, x2, y1, y2 in zip([1] * len(mean_rest), [2] * len(mean_tap), mean_rest, mean_tap):
    plt.plot([x1, x2], [y1, y2], 'k:', alpha=0.3)

plt.ylabel('Mean LFP Power 40-90Hz')
plt.legend(loc='upper right')
plt.xticks([1, 2], ['Rest', 'Tap'])
plt.title('Gamma Modulation Movement Med On-Stim Off')
plt.show()

In [ ]:
plt.savefig(os.path.join(
    dir,'MoveMod_StimOff'
), dpi = 200)

### Exemplary Patient Movement Modulation

In [ ]:
fft_npy = np.load(os.path.join(
    project_path,
    'data', 'FFTs', 'with_med_FTG',
    'Sub029_FFT.npy'
))

fft_npy.shape

In [ ]:
accel = np.load('S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all\\Synced_traces\\Sub029_cut_traces.npy')
accel.shape
import dat_preproc
faccel = dat_preproc.low_highpass_filter(accel, 1, 124)

In [ ]:
stim = np.full((1,323),1.4)

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(2,1)
ax2 = ax[0].twinx()
time_points = np.arange(accel.shape[1]) / 250

ax[0].pcolormesh(fft_npy[1,:,:], cmap = 'viridis', vmin = 0, vmax = 0.8) #vmin = -4, vmax = 7)
ax[0].set_xlim(100,200)
ax[0].set_ylim(60,80)
ax[0].set_xticklabels([])
ax[0].set_yticklabels([])
ax2.plot(np.arange(0, fft_npy.shape[2]), stim[0], color = 'white', linewidth = 2.5, linestyle = 'dotted')
ax2.set_yticks(np.arange(1,3,0.5))
ax2.set_yticklabels([])
ax[1].plot(time_points, faccel[2,:], color = 'grey', alpha = 0.8)
ax[1].set_xlim(100,200)
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
#plt.

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Sub029_ExemplaryMovementEmpty'),
    dpi=200)

'''plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Sub029_ExemplaryMovement.jpg'))'''

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import mne  # Import the mne module if not already imported

def save_pdf_without_antialiasing(file_path, fig, dpi=100):
    with PdfPages(file_path) as pdf:
        # Save the figure as a PDF without anti-aliasing
        pdf.savefig(fig, dpi=dpi)

if __name__ == "__main__":
    # Example usage
    # Replace 'figure.pdf' with the desired file path and name for your PDF file
    file_path = "figure.pdf"

    fig, ax = plt.subplots(2, 1, figsize=(8.5, 11))
    bs_data = mne.baseline.rescale(data=fft_npy[1, :, :], times=np.arange(0, fft_npy.shape[2]), baseline=baseline, mode='zscore')

    ax[0].pcolormesh(fft_npy[1, :, :], cmap='viridis', vmin=0, vmax=0.8)
    ax[0].set_xlim(100, 200)
    ax[0].set_ylim(60, 80)
    ax[0].set_ylabel('Frequency [Hz]')

    ax[1].plot(time_points, faccel[2, :], color='grey', alpha=0.8)
    ax[1].set_xlim(100, 200)
    ax[1].set_ylabel('Acceleration [g]')
    plt.xlabel('Time [sec]')

    save_pdf_without_antialiasing(file_path, fig)

    # Show the plot (optional)
    plt.show()


# Motor Performance with Increased Stim Amp

## 1. Write down in Excel all epochs and amplitudes

In [ ]:
# Load corresponding files
SUBID = 'Sub065'

#define your paths
fif_path = os.path.join(project_path, 'data', 'Fifs', 'with_med_FTG')
FFTs_path = os.path.join(project_path, 'data', 'FFTs', 'with_med_FTG')
cut_traces_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all\\Synced_traces\\'

#Import your data
accel = np.load(os.path.join(cut_traces_path, 'Sub065_cut_traces.npy'))
#accel = np.genfromtxt(os.path.join(cut_traces_path, 'Sub050_cut_traces.csv'), delimiter=',')

Draw = mne.io.read_raw_fif(os.path.join(fif_path,'Sub065_FIF.fif'))

Draw.ch_names

In [ ]:
%matplotlib qt
time_points1 = np.arange(accel.shape[1]) / 250
time_points2 = np.arange(Draw.n_times) / 250
stim = Draw.get_data(picks = 'STIM_L_125Hz_60us')[0]

plt.plot(time_points1, accel[2,:])
plt.plot(time_points2, stim) #*10e5

## 2. Identify all Features from blocks

In [ ]:
SUBID = 'Sub033'

blocks_dir = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\retap_results\\features\\'
#blocks_dir = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\without_med_FTG\\raw_files\\retap_results\\features'
blocks_epochs = pd.read_excel(os.path.join(
    project_path, 'results', 'accelerometer', 'Accel_Blocks_AllInBetween.xlsx'
))

all_files = os.listdir(blocks_dir)

json_files = [file for file in all_files if file.startswith(f'features_{SUBID}') and file.endswith('.json')]

for json_file in json_files:
    print(os.path.join(blocks_dir, json_file))


## 3. Average the Blocks with the same Amp together

In [ ]:
filtered_df = blocks_epochs[blocks_epochs['PerceptID'] == SUBID].dropna(subset=['Block'])
filtered_df['Block'] = filtered_df['Block'].astype('int')
blocks_with_same_amplitude = filtered_df.groupby('Amplitude')['Block'].unique().dropna().to_dict()

keys_to_average = ['coefVar_intraTapInt', 'trace_RMSn', 'coefVar_impactRMS', 'mean_raise_velocity']


for amplitude, blocks in blocks_with_same_amplitude.items():
    amplitude_blocks = [f'block{block}' for block in blocks]
    entrainment_value = filtered_df[filtered_df['Block'].isin(blocks)]['Entrainment'].tolist()[0]
    # Filter files that have the same block number
    relevant_files = [file for file in json_files if any(f'block{block}.json' in file for block in blocks)]

    averaged_values = {'SubID': SUBID, 'amplitude': amplitude, 'entrainment': entrainment_value}

    for key in keys_to_average:
        values = []

        # Extract values for the key from each relevant file
        for file in relevant_files:
            try:
                with open(os.path.join(blocks_dir, file), 'r') as f:
                    file_data = json.load(f)
                    if key in file_data:
                        values.append(file_data[key])
                    else:
                        values.append(None)  # Handle the case where the key is missing
            except FileNotFoundError as e:
                print(f"Error reading {file}: {e}")

        # Calculate the average for the key
        if values:
            averaged_values[key] = sum(values) / len(values)
        else:
            averaged_values[key] = None  
                
        amp_nondot = str(amplitude).replace('.', '')
        features_json_title = f'{SUBID}_AvgFeatures_Amp{amp_nondot}mA.json'
        json_avg_features = os.path.join(
            project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks',
            features_json_title
        )

        with open(json_avg_features, 'w') as json_file:
            json.dump(averaged_values, json_file)


## 4. Plot all together and inspect the effect of Entrainment

In [ ]:
avg_features = os.path.join(project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks')
all_jsons_avg_blocks = os.listdir(avg_features)
json_allblocks_thisSUB = [file for file in all_jsons_avg_blocks if file.startswith(SUBID) and file.endswith('.json')]
json_allblocks_thisSUB

In [ ]:
# Example key of interest
keys_of_interest = ['coefVar_intraTapInt', 'trace_RMSn', 'coefVar_impactRMS', 'mean_raise_velocity']

# Initialize a figure with subplots
fig, axs = plt.subplots(len(keys_of_interest), 1, figsize=(8, 2 * len(keys_of_interest)), sharex=True)

data_dict = {'SubID': SUBID}

# Iterate through each key of interest
for i, key_of_interest in enumerate(keys_of_interest):
    amplitudes = []
    values_of_interest = []
    entrainments = []
    colors = []

    # Iterate through each JSON file
    for json_file in json_allblocks_thisSUB:
        with open(os.path.join(avg_features, json_file), 'r') as f:
            file_data = json.load(f)

            # Check if the key of interest is present in the dictionary
            if key_of_interest in file_data:
                # Extract 'amplitude' value and the value of the key of interest
                amplitude = file_data.get('amplitude', None)
                value_of_interest = file_data[key_of_interest]
                entrainment = file_data.get('entrainment')

                # Append values to the lists
                amplitudes.append(amplitude)
                values_of_interest.append(value_of_interest)
                entrainments.append(entrainment)
                # Set color based on 'entrainment' value
                colors.append('red' if entrainment == 1 else 'blue')

    # Create a subplot for each key of interest
    axs[i].plot(amplitudes, values_of_interest, color='gray', linestyle='-', label='Values')
    axs[i].scatter(amplitudes, values_of_interest, c=colors, label='Values')
    axs[i].set_ylabel(f'{key_of_interest} Values')
    axs[i].set_ylabel(f'{key_of_interest}')
    
    # Create a dictionary for each key of interest
    key_data_dict = {
        'amplitudes': amplitudes,
        'values_of_interest': values_of_interest,
        'entrainments': entrainments
    }

    # Add the nested dictionary to the main dictionary
    data_dict[key_of_interest] = key_data_dict

axs[0].set_title(SUBID)
axs[-1].set_xlabel('Amplitude (mA)')

# Show the plot
plt.show()


### SAVE ALL
avg_features = os.path.join(project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks','All_Amplitudes')
json_all_avg = f'{SUBID}_AvgFeatures_AllAmps.json'
json_filename = os.path.join(avg_features,json_all_avg)
with open(json_filename, 'w') as json_file:
            json.dump(data_dict, json_file)

plt.savefig(os.path.join(avg_features,f'{SUBID}__AvgFeatures_AllAmps'), dpi = 150)

data_dict

In [ ]:
#Normalize Stimulation Intensity

amps_excel = pd.read_excel(os.path.join(project_path,'results','accelerometer','Accel_Blocks_AllInBetween.xlsx'),
                           sheet_name='Amplitudes')
amps_excel

In [ ]:
# Columns to normalize
columns_to_normalize = amps_excel.columns[1:8]

# Copy the DataFrame to avoid modifying the original
normalized_dataframe = amps_excel.copy()

# Iterate through each row and normalize the values
for index, row in normalized_dataframe.iterrows():
    for column in columns_to_normalize:
        # Calculate the percentage difference with respect to the value in 'On_Entrain' column
        normalized_value = (row[column] / row['On_Entrain']) * 100
        normalized_dataframe.at[index, column] = normalized_value



In [ ]:
normalized_dataframe

In [ ]:
motor_feat

In [ ]:
%matplotlib qt

avg_features = os.path.join(project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks','All_Amplitudes')

features = ['coefVar_intraTapInt', 'trace_RMSn', 'coefVar_impactRMS', 'mean_raise_velocity']


fig, axs = plt.subplots(4, 1, figsize=(8, 2 * 4), sharex=True)

for jk, this_feat in enumerate(features):

    data_sets = []
    
    for subID in normalized_dataframe['SubID']:
        SUBID = subID

        this_file = f'{SUBID}_AvgFeatures_AllAmps.json'

        json_file_path = os.path.join(avg_features, this_file)
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)

        idx_row = normalized_dataframe[normalized_dataframe['SubID'] == SUBID].index[0]
        norm_amps = normalized_dataframe.iloc[idx_row,1:7].dropna().tolist()

        motor_feat = np.array(data[this_feat]['values_of_interest'])
        #motor_feat = ((motor_feat - motor_feat[0]) / motor_feat[0]) * 100
        
        this_data_set = {'SubID': SUBID, 'norm_amps': norm_amps, 'motor_feat': motor_feat}
        data_sets.append(this_data_set)

        ### SAVE ALL
        json_datraset_allfeats = f'AllFeatDBSIntensity_{this_feat}.pkl'
        json_filename = os.path.join(avg_features,json_datraset_allfeats)
        with open(json_filename, 'wb') as f:
           pickle.dump(data_sets, f)
        
        '''for i, data in enumerate(data_sets):
            axs[jk].plot(data['norm_amps'], data['motor_feat'], label=f'{SUBID}', marker='o', linestyle='-')
            axs[jk].axvline(x=100, lw = 0.3, ls = ':', color = 'grey')
            axs[jk].set_ylabel(f'{this_feat}\n(% Change DBS Off)')
            axs[jk].set_xlabel('DBS Intensity [reltive to entrainment amplitude %]')
            #axs[jk].set_xticklabels(np.arange(-20, 160, 20))'''

In [ ]:
plt.savefig(os.path.join(avg_features,'AllFeatures_Plot'), dpi = 200)

In [ ]:
############## MOTOR FEATURES RELATIVE TO DBS OFF ##############

import seaborn as sns

avg_features = os.path.join(project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks','All_Amplitudes')

pickle_filenames = ['AllFeatDBSIntensity_mean_raise_velocity.pkl',
                    'AllFeatDBSIntensity_trace_RMSn.pkl',
                    'AllFeatDBSIntensity_coefVar_impactRMS.pkl',
                    'AllFeatDBSIntensity_coefVar_intraTapInt.pkl']

feature_names = ['Mean Raise Velocity [m/s]',
                 'Norm. Trace RMS',
                 'Coef. Var. Impact RMS',
                 'Coef. Var. Intra Tap Interval']

###### baseline correction function ######
def baseline_correct(group):
    baseline_value = group.loc[group['motor_feat_range'] == RANGE, 'motor_feat'].values
    group['motor_feat'] = (group['motor_feat'] - baseline_value) / baseline_value * 100
    return group
##########################################

RANGE = 0 #condition (see bins below) relative to which baseline correction will be applied

if RANGE == 0:
    ylabel = '(% Relative to DBS-Off)'
elif RANGE == 2:
    ylabel = '(% Relative to Pre-Entrain)'

fig, axes = plt.subplots(4, 1, figsize=(7, 9.5))

for kl, pkl_filename in enumerate(pickle_filenames):
    file_path = os.path.join(avg_features,pickle_filenames[kl])
    with open(file_path, 'rb') as f:
        my_dict = pickle.load(f)

    # Unpack the dictionaries
    norm_amps = []
    motor_feat = []
    sub_ids = []
    for entry in my_dict:
        sub_id = entry['SubID']
        for amp, feat in zip(entry['norm_amps'], entry['motor_feat']):
            norm_amps.append(amp)
            motor_feat.append(feat)
            sub_ids.append(sub_id)

    df_unpacked = pd.DataFrame({'SubID': sub_ids, 'norm_amp': norm_amps, 'motor_feat': motor_feat})

    # Correct the normalized amplitudes into bins & average
    bins = [0, 20, 60, 90, 101, float('inf')]
    labels = [0,1,2,3,4]
    df_unpacked['motor_feat_range'] = pd.cut(df_unpacked['norm_amp'], bins=bins, labels=labels, include_lowest=True)
    df_unpacked_avg = df_unpacked.groupby(['SubID', 'motor_feat_range']).agg({'norm_amp': 'mean', 'motor_feat': 'mean'}).reset_index()
    df_unpacked_avg['motor_feat_change'] = df_unpacked_avg.groupby('SubID')['motor_feat'].pct_change()
    df_unpacked_avg = df_unpacked_avg.dropna(subset=['motor_feat_change'])
    
    # Apply baseline correction for each subject
    #baseline_corrected_df = df_unpacked_avg.groupby('SubID').apply(baseline_correct)
    
    # Boxplots    
    DATAFRAME = df_unpacked_avg
    
    ax = axes[kl]  # Select the appropriate subplot
    y = 'motor_feat_change'
    boxplot = sns.boxplot(x='motor_feat_range', y=y, data=DATAFRAME, 
                          width=0.5, showfliers=False, color='white', ax=ax)
    colors = sns.color_palette('viridis', len(DATAFRAME['SubID'].unique()))

    # Adjust marker size for scatter plot
    for sub_id, color in zip(DATAFRAME['SubID'].unique(), colors):
        sub_data = DATAFRAME[DATAFRAME['SubID'] == sub_id]
        sub_data_sorted = sub_data.sort_values(by='motor_feat_range')
        ax.scatter(sub_data_sorted['motor_feat_range'], sub_data_sorted[y], color=color, label=sub_id, s=40)  # Increase marker size

    #ax.set_xlabel('DBS Intensity (% relative to Amp of Entrainment Onset)')
    #ax.set_xticklabels(['0% (DBS Off)','25-60%','60-90% \n (Pre-Entrain)','100% \n (Entrain Onset)','>100%'])
    ax.set_xticklabels(['0% (DBS Off)','25-60%','60-90% \n (Pre-Entrain)','100% \n (Entrain Onset)','>100%'])
    ax.set_ylabel(f'{feature_names[kl]} \n {ylabel}')
    #ax.legend()

    # Set x-label for the last subplot
    if kl == len(pickle_filenames) - 1:
        ax.set_xlabel('DBS Intensity \n (% relative to Amp of Entrainment Onset)', labelpad=20)
    else:
        ax.set_xlabel('')

    plt.tight_layout()
    plt.show()


In [ ]:
figname = 'All_features_RelativeOff'
plt.savefig(os.path.join(avg_features,figname), dpi=200)

In [ ]:
############## MOTOR FEATURES RELATIVE TO PRE-ENTRAIN ##############

def baseline_correctpreEntrain(group):
    baseline_value = None
    
    # Check for motor_feat_range == 2
    if (group['motor_feat_range'] == 2).any():
        non_nan_values = group.loc[group['motor_feat_range'] == 2, 'motor_feat'].dropna()
        if not non_nan_values.empty:
            baseline_value = non_nan_values.values[0]

    # Check for motor_feat_range == 1 if baseline_value is still None
    if baseline_value is None and (group['motor_feat_range'] == 1).any():
        non_nan_values = group.loc[group['motor_feat_range'] == 1, 'motor_feat'].dropna()
        if not non_nan_values.empty:
            baseline_value = non_nan_values.values[0]
    
    if baseline_value is not None:
        group['motor_feat_Corr'] = (group['motor_feat'] - baseline_value) / baseline_value * 100
    else:
        group['motor_feat_Corr'] = np.nan  # Mark all values as NaN
        
    return group

avg_features = os.path.join(project_path, 'results', 'accelerometer', 'Avg_Features_allBlocks','All_Amplitudes')

pickle_filenames = ['AllFeatDBSIntensity_mean_raise_velocity.pkl',
                    'AllFeatDBSIntensity_trace_RMSn.pkl',
                    'AllFeatDBSIntensity_coefVar_impactRMS.pkl',
                    'AllFeatDBSIntensity_coefVar_intraTapInt.pkl']

feature_names = ['Mean Raise Velocity',
                 'Norm. Trace RMS',
                 'Coef. Var. Impact RMS',
                 'Coef. Var. Intra Tap Interval']

fig, axes = plt.subplots(1,4, figsize=(15,5))

for kl, pkl_filename in enumerate(pickle_filenames):
    file_path = os.path.join(avg_features,pickle_filenames[kl])
    with open(file_path, 'rb') as f:
        my_dict = pickle.load(f)

    # Unpack the dictionaries
    norm_amps = []
    motor_feat = []
    sub_ids = []
    for entry in my_dict:
        sub_id = entry['SubID']
        for amp, feat in zip(entry['norm_amps'], entry['motor_feat']):
            norm_amps.append(amp)
            motor_feat.append(feat)
            sub_ids.append(sub_id)

    df_unpacked = pd.DataFrame({'SubID': sub_ids, 'norm_amp': norm_amps, 'motor_feat': motor_feat})

    # Correct the normalized amplitudes into bins & average
    bins = [0, 20, 60, 90, 101, float('inf')]
    labels = [0,1,2,3,4]
    df_unpacked['motor_feat_range'] = pd.cut(df_unpacked['norm_amp'], bins=bins, labels=labels, include_lowest=True)
    #df_unpacked_avg = df_unpacked.groupby(['SubID', 'motor_feat_range']).agg({'norm_amp': 'mean', 'motor_feat': 'mean'}).reset_index()
    
    baseline_correctedPreEntrain_df = df_unpacked.groupby('SubID').apply(baseline_correctpreEntrain)
    
    string_without_gaps = feature_names[kl].replace(" ", "")
    #baseline_correctedPreEntrain_df.to_excel(os.path.join(
    #    avg_features, f'{string_without_gaps}.xlsx'
    #), index=None)
    
    plot_entr = baseline_correctedPreEntrain_df['motor_feat_Corr'][baseline_correctedPreEntrain_df['norm_amp'] == 100.0].dropna()
    
    ax = axes[kl]
    ax.boxplot(plot_entr)
    ax.scatter(np.ones((1, len(plot_entr))),plot_entr)
    ax.axhline(y=0, color='grey', linestyle=':')
    ax.set_xticklabels(['Stim Amp of Entrainment Onset'])
    ax.set_title(f'{feature_names[kl]} \n % Rel. Stim. Amplitude \n Pre Entrain')
    plt.show()


In [ ]:
figname = 'All_features_RelativePreEntrainment'
plt.savefig(os.path.join(avg_features,figname), dpi=200)

# Dyskinesia Scores

In [ ]:
path_dysk_ratings = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Ratings'


for filename in os.listdir(path_dysk_ratings):
    if filename.endswith('.xlsx'):
        

        rate_df = pd.read_excel(os.path.join(
            path_dysk_ratings, filename
        ), sheet_name='Dataset')
        rate_df['Ipsi_Delta'] = rate_df['Ipsi'] - rate_df['Ipsi'][0]
        rate_df['Contra_Delta'] = rate_df['Contra'] - rate_df['Contra'][0]
        rate_df['Total_Delta'] = rate_df['Total'] - rate_df['Total'][0]
        
        SUBID = filename.split('.')[0]
        
        %matplotlib qt

        fig, axes = plt.subplots(1,2, figsize=(12,6))

        if any(rate_df['Reporting'] == 1):
            side_eff_onset1 = rate_df.loc[rate_df['Reporting'] == 1, 'Amplitude'].tolist()[0]
            side_eff_onset2 = rate_df['Amplitude_Norm'][rate_df['Reporting'] == 1].tolist()[0]
            
        axes[0].scatter(rate_df['Amplitude'], rate_df['Contra'], marker='o', label = 'Contralateral CDRS')
        axes[0].scatter(rate_df['Amplitude'], rate_df['Ipsi'], label = 'Ipsilateral CDRS')
        axes[0].scatter(rate_df['Amplitude'], rate_df['Total'], label = 'Total CDRS')
        axes[0].set_xlabel('DBS Amplitude (mA)')
        axes[0].set_ylabel('CDRS Score')
        axes[0].axvline(entrain_onset1, color = 'grey', ls = ':', label = 'Entrainment Onset')
        axes[0].axvline(side_eff_onset1, color = 'magenta', ls = ':', label = 'Side-Effect Reporting')
        axes[0].legend()


        entrain_onset2 = rate_df['Amplitude_Norm'][rate_df['Amplitude_Norm'] == 100].tolist()[0]
        axes[1].scatter(rate_df['Amplitude_Norm'], rate_df['Contra_Delta'], marker='o', label = 'Contralateral CDRS')
        axes[1].scatter(rate_df['Amplitude_Norm'], rate_df['Ipsi_Delta'], label = 'Ipsilateral CDRS')
        axes[1].scatter(rate_df['Amplitude_Norm'], rate_df['Total_Delta'], label = 'Total CDRS')
        axes[1].set_xlabel('DBS Intensity (% Rel to Amp of Entrainment Onset)')
        axes[1].set_ylabel('CDRS Score (Delta Rel to DBS Off)')
        axes[1].axvline(entrain_onset2, color = 'grey', ls = ':', label = 'Entrainment Onset')
        axes[1].axvline(side_eff_onset2, color = 'magenta', ls = ':', label = 'Side-Effect Reporting')
        axes[1].legend()

        plt.suptitle(SUBID)
        
        plt.savefig(os.path.join(
            project_path, 'figures', 'CDRS', f'{SUBID}_CDRS'
        ), dpi = 150)


In [ ]:
## Group Figure
import random
path_dysk_ratings = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Ratings'

fig, axes = plt.subplots(figsize=(8,6))

dict_cohorts = {
    'cohort_with' : ['Sub005', 'Sub006','Sub009','Sub014','Sub017','Sub029','Sub045','Sub065'],
    'cohort_without' : ['Sub007', 'Sub021','Sub025','Sub028','Sub033','Sub043','Sub050']
}

lines_with = []
lines_without = []
entrain_lines = []

for filename in os.listdir(path_dysk_ratings):
    random_number = random.uniform(-0.7, 0)
    if filename.endswith('.xlsx'):
        rate_df = pd.read_excel(os.path.join(path_dysk_ratings, filename), sheet_name='Dataset')
        #rate_df['Ipsi_Delta'] = rate_df['Ipsi'] - rate_df['Ipsi'][0]
        #rate_df['Contra_Delta'] = rate_df['Contra'] - rate_df['Contra'][0]
        #rate_df['Total_Delta'] = rate_df['Total'] - rate_df['Total'][0]
        if rate_df['Total'][0] == 0:
            rate_df['Total'] = rate_df['Total'] + random_number
        SUBID = filename.split('.')[0]
        
        rate_df['Amp_Dev'] = rate_df['Amplitude'] - rate_df.loc[rate_df['Amplitude_Norm'] == 100, 'Amplitude'].values[0]
        #rate_df.loc[rate_df['Amplitude_Norm'] == 0, 'Amp_Dev'] = 0

        entrain_onset2 = rate_df['Amp_Dev'][rate_df['Amp_Dev'] == 0].tolist()[0]
        entrain_line = plt.axvline(entrain_onset2, color='grey', ls=':', label = 'Entrainment Onset')
        if SUBID in dict_cohorts['cohort_with']:
            color='#88CCEE'
            line, = plt.plot(rate_df['Amp_Dev'], rate_df['Total'], color=color, label='with FTG DBS-Off', lw = 2)
            lines_with.append(line)
            entrain_lines.append(entrain_line)
        elif SUBID in dict_cohorts['cohort_without']:
            color='#AA4499' 
            line, = plt.plot(rate_df['Amp_Dev'], rate_df['Total'], color=color,label='no FTG DBS-Off', lw = 2)
            lines_without.append(line)
        
        
    
axes.legend(handles=[lines_with[0], lines_without[0], entrain_line], labels=['with FTG DBS-Off (n=8)', 'no FTG DBS-Off (n=7)','Entrainment Onset'])
plt.xlabel('DBS Intensity (mA Rel to Amp of Entrainment Onset)')
plt.ylabel('Total CDRS Score')
#axes.set_xticklabels(['','0% (DBS-Off)','50%','100% \n Entrain Onset','150%','200%','250%'])
axes.set_yticks(np.arange(0,13,2))
plt.show()


In [ ]:
plt.savefig(os.path.join(
            project_path, 'figures', 'CDRS', 'Total_CDRS_GroupFigure_Relative'
        ), dpi = 150)